## jieba中文处理

和拉丁语系不同，亚洲语言是不用空格分开每个有意义的词的。而当我们进行自然语言处理的时候，大部分情况下，词汇是我们对句子和文章理解的基础，因此需要一个工具去把完整的文本中分解成粒度更细的词。
jieba就是这样一个非常好用的中文工具，是以分词起家的，但是功能比分词要强大很多。
### 基本分词函数与用法
**jieba.cut** 以及 **jieba.cut_for_search** 返回的结构都是一个可迭代的 generator，可以使用 for 循环来获得分词后得到的每一个词语(unicode)

**jieba.cut** 方法接受三个输入参数:
- 需要分词的字符串
- cut_all 参数用来控制是否采用全模式
- HMM 参数用来控制是否使用 HMM 模型

**jieba.cut_for_search** 方法接受两个参数
- 需要分词的字符串
- 是否使用 HMM 模型。
- 该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细

In [28]:
# encoding=utf-8
import jieba
cut_list = jieba.cut("我在学习自然语言处理计算机视觉",cut_all=True) #全模式
print(cut_list)
print("Full mode" + '/'.join(cut_list))
cut_list_1 = jieba.cut("我在学习自然语言处理，计算机视觉",cut_all=False) #精确模式，默认是精确模式
print("Default mode" + '/'.join(cut_list_1))
cut_list_2 = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在哈佛大学深造") #搜索引擎模式
print('/'.join(cut_list_2))
cut_list_2 = jieba.cut_for_search("科比巅峰时到底能不能虐詹姆斯，这不是问题") #搜索引擎模式
print('/'.join(cut_list_2))

<generator object Tokenizer.cut at 0x10cadd990>
Full mode我/在/学习/自然/自然语言/语言/处理/计算/计算机/算机/视觉
Default mode我/在/学习/自然语言/处理/，/计算机/视觉
小明/硕士/毕业/于/中国/科学/学院/科学院/中国科学院/计算/计算所/，/后/在/哈佛/大学/哈佛大学/深造
科比/巅峰/时/到底/能/不能/虐/詹姆斯/，/这/不是/问题


**jieba.lcut**以及**jieba.lcut_for_search**直接返回 list

In [29]:
result_lcut = jieba.lcut("小明硕士毕业于中国科学院计算所，后在哈佛大学深造")
print(result_lcut)
print(" ".join(result_lcut))
print(" ".join(jieba.lcut_for_search("小明硕士毕业于中国科学院计算所，后在哈佛大学深造")))

['小明', '硕士', '毕业', '于', '中国科学院', '计算所', '，', '后', '在', '哈佛大学', '深造']
小明 硕士 毕业 于 中国科学院 计算所 ， 后 在 哈佛大学 深造
小明 硕士 毕业 于 中国 科学 学院 科学院 中国科学院 计算 计算所 ， 后 在 哈佛 大学 哈佛大学 深造


### 添加用户自定义词典
很多时候我们需要针对自己的场景进行分词，会有一些领域内的专有词汇。
- 1.可以用jieba.load_userdict(file_name)加载用户字典
- 2.少量的词汇可以自己用下面方法手动添加：
  - 用 add_word(word, freq=None, tag=None) 和 del_word(word) 在程序中动态修改词典
  - 用 suggest_freq(segment, tune=True) 可调节单个词语的词频，使其能（或不能）被分出来。

In [30]:
print('/'.join(jieba.cut('如果放到旧字典中将出错。', HMM=False)))

如果/放到/旧/字典/中将/出错/。


In [31]:
jieba.suggest_freq(('中', '将'), True)

494

In [32]:
print('/'.join(jieba.cut('如果放到旧字典中将出错。', HMM=False)))

如果/放到/旧/字典/中/将/出错/。


### 并行分词
原理：将目标文本按行分隔后，把各行文本分配到多个 Python 进程并行分词，然后归并结果，从而获得分词速度的可观提升 基于 python 自带的 multiprocessing 模块，(是否支持windows待验证)

用法：
   jieba.enable_parallel(4) # 开启并行分词模式，参数为并行进程数
   jieba.disable_parallel() # 关闭并行分词模式

实验结果：在 4 核 3.4GHz Linux 机器上，对金庸全集进行精确分词，获得了 1MB/s 的速度，是单进程版的 3.3 倍。

注意：并行分词仅支持默认分词器 jieba.dt 和 jieba.posseg.dt

In [53]:
import sys
import time
import jieba

jieba.enable_parallel()
content = open(u'人工智能火了-人类有些尴尬了.txt',"r").read()
t1 = time.time()
words = "/ ".join(jieba.cut(content))
t2 = time.time()
tm_cost = t2-t1
print('并行分词速度为 %s bytes/second' % (len(content)/tm_cost))

jieba.disable_parallel()
content = open(u'人工智能火了-人类有些尴尬了.txt',"r").read()
t1 = time.time()
words = "/ ".join(jieba.cut(content))
t2 = time.time()
tm_cost = t2-t1
print('非并行分词速度为 %s bytes/second' % (len(content)/tm_cost))

并行分词速度为 240343.8391779829 bytes/second
非并行分词速度为 185585.00661728394 bytes/second


### 关键词提取¶
#### 基于 TF-IDF 算法的关键词抽取
import jieba.analyse
 - jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
   - sentence 为待提取的文本
   - topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
   - withWeight 为是否一并返回关键词权重值，默认值为 False
   - allowPOS 仅包括指定词性的词，默认值为空，即不筛选

In [46]:
import jieba.analyse as analyse
lines = open('人工智能火了-人类有些尴尬了.txt').read()
print(type(lines))
seg_list = analyse.extract_tags(lines,topK=15,withWeight=False,allowPOS=())
#print(seg_list)
text = " ".join(seg_list)
print(text)

<class 'str'>
人工智能 人类 伦理 法律 张钹 真真 索菲亚 隐私 学习 人机 AlphaGo 系统 智能 机器 深度


#### 关于TF-IDF 算法的关键词抽取补充
- 关键词提取所使用逆向文件频率（IDF）文本语料库可以切换成自定义语料库的路径
  - 用法： jieba.analyse.set_idf_path(file_name) # file_name为自定义语料库的路径
- 关键词提取所使用停止词（Stop Words）文本语料库可以切换成自定义语料库的路径
  - 用法： jieba.analyse.set_stop_words(file_name) # file_name为自定义语料库的路径

#### 基于 TextRank 算法的关键词抽取¶
- jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。
- jieba.analyse.TextRank() 新建自定义 TextRank 实例
##### 基本思想:
- 将待抽取关键词的文本进行分词
- 以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图
- 计算图中节点的PageRank，注意是无向带权图

In [50]:
import jieba.analyse as analyse
lines = open('人工智能火了-人类有些尴尬了.txt').read()
print("  ".join(analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))
print("---------------------我是分割线----------------")
print("  ".join(analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n'))))

人工智能  人类  法律  伦理  系统  学习  发展  机器  智能  问题  存在  科学  没有  技术  深度  人机  带来  概念  不可  使用
---------------------我是分割线----------------
人工智能  人类  法律  伦理  系统  机器  智能  科学  结果  问题  概念  技术  深度  规范  科技  产业界  方法  人机  目标  上学


### 词性标注
- jieba.posseg.POSTokenizer(tokenizer=None) 新建自定义分词器，tokenizer 参数可指定内部使用的 jieba.Tokenizer 分词器。jieba.posseg.dt 为默认词性标注分词器。
- 标注句子分词后每个词的词性，采用和 ictclas 兼容的标记法。
- 具体的词性对照表参见计算所汉语词性标记集

In [52]:
import jieba.posseg as pseg
words = pseg.cut("我爱人工智能")
print(words)
for word,flag in words:
    print("%s %s" % (word,flag))

<generator object cut at 0x111680eb8>
我 r
爱 v
人工智能 n


### Tokenize：返回词语在原文的起止位置
注意，输入参数只接受 unicode

In [57]:
print("这是默认模式的tokenize")
result = jieba.tokenize(u'自然语言处理非常有用')
for tk in result:
    print("%s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

print(u"\n-----------我是神奇的分割线------------\n")

print("这是搜索模式的tokenize")
result = jieba.tokenize(u'自然语言处理非常有用', mode='search')
for tk in result:
    print("%s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

这是默认模式的tokenize
自然语言		 start: 0 		 end:4
处理		 start: 4 		 end:6
非常		 start: 6 		 end:8
有用		 start: 8 		 end:10

-----------我是神奇的分割线------------

这是搜索模式的tokenize
自然		 start: 0 		 end:2
语言		 start: 2 		 end:4
自然语言		 start: 0 		 end:4
处理		 start: 4 		 end:6
非常		 start: 6 		 end:8
有用		 start: 8 		 end:10


### ChineseAnalyzer for Whoosh 搜索引擎
from jieba.analyse import ChineseAnalyzer
待验证

In [ ]:
命令行分词
使用示例：python -m jieba news.txt > cut_result.txt
命令行选项（翻译）：
使用: python -m jieba [options] filename

结巴命令行界面。

固定参数:
  filename              输入文件

可选参数:
  -h, --help            显示此帮助信息并退出
  -d [DELIM], --delimiter [DELIM]
                        使用 DELIM 分隔词语，而不是用默认的' / '。
                        若不指定 DELIM，则使用一个空格分隔。
  -p [DELIM], --pos [DELIM]
                        启用词性标注；如果指定 DELIM，词语和词性之间
                        用它分隔，否则用 _ 分隔
  -D DICT, --dict DICT  使用 DICT 代替默认词典
  -u USER_DICT, --user-dict USER_DICT
                        使用 USER_DICT 作为附加词典，与默认词典或自定义词典配合使用
  -a, --cut-all         全模式分词（不支持词性标注）
  -n, --no-hmm          不使用隐含马尔可夫模型
  -q, --quiet           不输出载入信息到 STDERR
  -V, --version         显示版本信息并退出

如果没有指定文件名，则使用标准输入。

--help 选项输出：
$> python -m jieba --help
Jieba command line interface.

positional arguments:
  filename              input file

optional arguments:
  -h, --help            show this help message and exit
  -d [DELIM], --delimiter [DELIM]
                        use DELIM instead of ' / ' for word delimiter; or a
                        space if it is used without DELIM
  -p [DELIM], --pos [DELIM]
                        enable POS tagging; if DELIM is specified, use DELIM
                        instead of '_' for POS delimiter
  -D DICT, --dict DICT  use DICT as dictionary
  -u USER_DICT, --user-dict USER_DICT
                        use USER_DICT together with the default dictionary or
                        DICT (if specified)
  -a, --cut-all         full pattern cutting (ignored with POS tagging)
  -n, --no-hmm          don't use the Hidden Markov Model
  -q, --quiet           don't print loading messages to stderr
  -V, --version         show program's version number and exit

If no filename specified, use STDIN instead.